In [1]:
import os
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

import collections

/opt/anaconda3/envs/bumblebee/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

04/23/2021 10:43:59 - INFO - tensorflow -   Enabling eager execution
04/23/2021 10:43:59 - INFO - tensorflow -   Enabling v2 tensorshape
04/23/2021 10:43:59 - INFO - tensorflow -   Enabling resource variables
04/23/2021 10:43:59 - INFO - tensorflow -   Enabling tensor equality
04/23/2021 10:43:59 - INFO - tensorflow -   Enabling control flow v2
04/23/2021 10:44:00 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Devices available: {}".format(device))

Devices available: cpu


In [4]:
tokenizer = Tokenizer.load(pretrained_model_name_or_path="bert-base-uncased")
#tokenizer = simple_preprocess

04/23/2021 10:44:01 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'BertTokenizer'


In [5]:
#LABEL_LIST = ["OTHER", "OFFENSE"]
LABEL_LIST = ['0','1']
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        data_dir="",
                                        train_filename="",
                                        test_filename="",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        label_column_name="")

In [6]:
save_dir = ''

In [8]:
def get_metrics(result, targets, comments):
    inputs = []
    preds = []
    probs = []
    count = 0
    for task in result:
        for pred in task['predictions']:
            inputs.append(pred['context'])
            #preds.append(int(pred['label']))
            probs.append(pred['probability'])
            preds.append(np.argmax(pred['probability']))
            #if int(pred['label']) != targets[count]:
            #    print(comments[count],"\t",pred['probability'])
            count += 1
    return preds,probs
    

In [ ]:
import transformers
import nlp
import torch
import numpy as np
import scipy as sp
import shap

In [ ]:
save_dir = ''
lang_model = LanguageModel.load(save_dir)

In [ ]:
tokenizer = Tokenizer.load(pretrained_model_name_or_path="bert-base-uncased",max_length=512)
model = AdaptiveModel.load(save_dir, device=device)

In [ ]:
count = 0
num_exmpls = 100

model = AdaptiveModel.load(save_dir, device=device)

# define a prediction function
def f(x):

    global count
    
    scores_vals = []
    for v in x:
        
        tkns = tokenizer(v)

        input_ids = torch.tensor([tkns['input_ids'][:512]])
        
        token_type_ids = torch.tensor([tkns['token_type_ids'][:512]])
        attention_mask = torch.tensor([tkns['attention_mask'][:512]])
        
        p = model(input_ids=input_ids,segment_ids=None,padding_mask=None,token_type_ids=token_type_ids,attention_mask=attention_mask)

        #print(p)
        outputs = p[0].detach().numpy()
        scores = np.exp(outputs) / np.exp(outputs).sum(-1)
        #print(scores)
        scores_vals.append(scores[0][1])
        #break;
    count += 1
    return scores_vals



In [16]:
# build an explainer using a token masker
explainer = shap.Explainer(f, tokenizer)

In [18]:
exmpl_coms = ["I hate you. I am going to kill you.","I really love working with him!"]

In [19]:
shap_values = explainer(exmpl_coms, fixed_context=1)

In [21]:
test_toxic_dataset = [{"text": x} for x in exmpl_coms]
result_test_toxic = infer_model.inference_from_dicts(dicts=test_toxic_dataset)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  6.40 Batches/s]


In [20]:
shap_values

.values =
array([array([-3.16384360e-05,  9.16245602e-02,  9.31230209e-02,  9.26267825e-02,
               9.05397457e-02,  8.93920093e-02,  8.93720417e-02,  8.90019863e-02,
               8.91698926e-02,  9.29179072e-02,  9.13188934e-02,  8.91513705e-02,
              -3.73100029e-04])                                                  ,
       array([-3.26508869e-05, -1.62628788e-03, -1.93587298e-03, -2.95698977e-03,
              -2.46217412e-03,  2.77337166e-03,  3.07033891e-03,  2.92569219e-03,
               5.18689760e-06])                                                  ],
      dtype=object)

.base_values =
array([0.00086607, 0.00053031])

.data =
array([list(['', 'I ', 'hate ', 'you', '. ', 'I ', 'am ', 'going ', 'to ', 'kill ', 'you', '.', '']),
       list(['', 'I ', 'really ', 'love ', 'working ', 'with ', 'him', '!', ''])],
      dtype=object)

In [21]:
print(shap_values.values[0])
print(shap_values.data[0])
inds = np.argsort(-shap_values.values[0])
for i in inds:
    print(shap_values.data[0][i],end=' ')

[-3.16384360e-05  9.16245602e-02  9.31230209e-02  9.26267825e-02
  9.05397457e-02  8.93920093e-02  8.93720417e-02  8.90019863e-02
  8.91698926e-02  9.29179072e-02  9.13188934e-02  8.91513705e-02
 -3.73100029e-04]
['', 'I ', 'hate ', 'you', '. ', 'I ', 'am ', 'going ', 'to ', 'kill ', 'you', '.', '']
hate  kill  you I  you .  I  am  to  . going    

In [22]:
# plot the first sentence's explanation
shap.plots.text(shap_values[0])

In [23]:
print(shap_values.values[1])
print(shap_values.data[1])
inds = np.argsort(shap_values.values[1])
for i in inds:
    print(shap_values.data[1][i],end=' ')

[-3.26508869e-05 -1.62628788e-03 -1.93587298e-03 -2.95698977e-03
 -2.46217412e-03  2.77337166e-03  3.07033891e-03  2.92569219e-03
  5.18689760e-06]
['', 'I ', 'really ', 'love ', 'working ', 'with ', 'him', '!', '']
love  working  really  I    with  ! him 

In [24]:
shap.plots.text(shap_values[1])